164402 - Andrey Vinicius Santos Souza

In [1]:
import numpy as np

def arredondar(num, sig=6):
    """Arredonda um número para um número fixo de algarismos significativos."""
    if num == 0:
        return 0.0
    else:
        return float(f"{num:.{sig}g}")

def arredondar_matriz(mat, sig=6):
    """Aplica arredondamento a todos os elementos de uma matriz."""
    return np.vectorize(lambda x: arredondar(x, sig))(mat)

def substituicao_regressiva(Ab, sig=6):
    """Executa substituição regressiva para resolver o sistema triangular superior."""
    n = len(Ab)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        soma = sum(arredondar(Ab[i, j] * x[j], sig) for j in range(i+1, n))
        x[i] = arredondar((Ab[i, -1] - soma) / Ab[i, i], sig)
        print(f"\n🔻 Substituição regressiva para x{i+1}: {x[i]:.8f}")
    return x

def eliminacao_gauss_pivotamento(A, b, sig=6):
    """Executa a Eliminação de Gauss com pivotamento parcial e imprime cada passo."""
    n = len(A)
    Ab = np.hstack((A, b.reshape(-1, 1)))  # Matriz aumentada

    print("\n🔢 Matriz aumentada inicial:")
    print(arredondar_matriz(Ab, sig))

    # Eliminação com pivotamento parcial
    for i in range(n):
        print(f"\n🔄 Passo {i+1} - Início da iteração")

        # Pivotamento parcial
        max_idx = np.argmax(np.abs(Ab[i:, i])) + i
        if i != max_idx:
            Ab[[i, max_idx]] = Ab[[max_idx, i]]
            print(f"\n🔁 Trocando linha {i+1} com linha {max_idx+1} (pivotamento parcial):")
            print(arredondar_matriz(Ab, sig))
        else:
            print(f"\n🔁 Nenhuma troca de linhas necessária.")

        # Eliminação de Gauss (para zerar abaixo do pivô)
        for j in range(i+1, n):
            if Ab[i, i] == 0:
                print(f"⚠️ Pivô nulo na linha {i+1}, não é possível continuar.")
                continue
            fator = arredondar(Ab[j, i] / Ab[i, i], sig)
            print(f"\n🔨 Calculando fator de eliminação: {fator} para a linha {j+1}")

            for k in range(i, n+1):
                Ab[j, k] = arredondar(Ab[j, k] - fator * Ab[i, k], sig)

            print(f"\n📊 Após eliminação da linha {j+1}:")
            print(arredondar_matriz(Ab, sig))

    print("\n📐 Matriz triangular superior:")
    print(arredondar_matriz(Ab, sig))

    # Substituição regressiva
    x = substituicao_regressiva(Ab, sig)

    print("\n✅ Solução final (com 8 casas decimais):")
    for i, valor in enumerate(x, 1):
        print(f"x{i} = {valor:.8f}")

    return x

def entrada_usuario():
    """Faz a entrada interativa da matriz e vetor."""
    try:
        tamanho = input("Digite o tamanho da matriz no formato 'nxn' (ex: 3x3): ").lower().strip()
        n = int(tamanho.split('x')[0])

        A = np.zeros((n, n))
        b = np.zeros(n)

        print("\n📥 Preenchendo a matriz A (coeficientes do sistema):")
        for i in range(n):
            for j in range(n):
                A[i, j] = float(input(f"  a{i+1}{j+1} = "))

        print("\n📥 Preenchendo o vetor b (termos independentes):")
        for i in range(n):
            b[i] = float(input(f"  b{i+1} = "))

        return A, b

    except Exception as e:
        print(f"\n❌ Erro de entrada: {e}")
        return None, None

# Execução principal
A, b = entrada_usuario()
if A is not None and b is not None:
    eliminacao_gauss_pivotamento(A, b, sig=6)


Digite o tamanho da matriz no formato 'nxn' (ex: 3x3): 3

📥 Preenchendo a matriz A (coeficientes do sistema):
  a11 = 2
  a12 = 1
  a13 = -1
  a21 = -3
  a22 = 4
  a23 = 2
  a31 = 1
  a32 = 2
  a33 = 3

📥 Preenchendo o vetor b (termos independentes):
  b1 = 120
  b2 = -20
  b3 = 90

🔢 Matriz aumentada inicial:
[[  2.   1.  -1. 120.]
 [ -3.   4.   2. -20.]
 [  1.   2.   3.  90.]]

🔄 Passo 1 - Início da iteração

🔁 Trocando linha 1 com linha 2 (pivotamento parcial):
[[ -3.   4.   2. -20.]
 [  2.   1.  -1. 120.]
 [  1.   2.   3.  90.]]

🔨 Calculando fator de eliminação: -0.666667 para a linha 2

📊 Após eliminação da linha 2:
[[-3.00000e+00  4.00000e+00  2.00000e+00 -2.00000e+01]
 [-1.00000e-06  3.66667e+00  3.33334e-01  1.06667e+02]
 [ 1.00000e+00  2.00000e+00  3.00000e+00  9.00000e+01]]

🔨 Calculando fator de eliminação: -0.333333 para a linha 3

📊 Após eliminação da linha 3:
[[-3.00000e+00  4.00000e+00  2.00000e+00 -2.00000e+01]
 [-1.00000e-06  3.66667e+00  3.33334e-01  1.06667e+02]
 [ 

## Sistema Resolvido com Método de Gauss

Os valores aproximados obtidos com 8 casas decimais são:

$$
\begin{aligned}
x &= 43{,}24330000 \\
y &= 29{,}45950000 \\
z &= -4{,}05409000
\end{aligned}
$$


In [2]:
import numpy as np
import sympy as sp

np.set_printoptions(precision=8, suppress=True)

def lu_com_pivotamento_parcial(A):
    n = A.shape[0]
    L = np.eye(n)
    U = A.copy().astype(np.float64)
    P = np.eye(n)

    for i in range(n):
        # Pivotamento parcial
        max_row = np.argmax(np.abs(U[i:, i])) + i
        if i != max_row:
            U[[i, max_row], :] = U[[max_row, i], :]
            P[[i, max_row], :] = P[[max_row, i], :]
            if i > 0:
                L[[i, max_row], :i] = L[[max_row, i], :i]

        for j in range(i+1, n):
            if U[i, i] == 0:
                raise ZeroDivisionError("Pivô zero encontrado!")
            fator = U[j, i] / U[i, i]
            L[j, i] = fator
            U[j, :] -= fator * U[i, :]

    return P, L, U

def resolver_sistema_lu(P, L, U, b):
    # Aplica permutação ao vetor b
    Pb = P @ b

    # Substituição progressiva para resolver L*y = Pb
    n = len(b)
    y = np.zeros(n)
    for i in range(n):
        y[i] = Pb[i] - np.dot(L[i, :i], y[:i])

    # Substituição regressiva para resolver U*x = y
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]

    return x

def entrada_matriz():
    n = int(input("Digite o tamanho da matriz no formato 'n' para nxn: "))
    A = np.zeros((n, n))
    b = np.zeros(n)

    print("\n📥 Preenchendo a matriz A:")
    for i in range(n):
        for j in range(n):
            A[i, j] = float(input(f"  a{i+1}{j+1}: "))

    print("\n📥 Preenchendo o vetor b:")
    for i in range(n):
        b[i] = float(input(f"  b{i+1}: "))

    return A, b

# Execução principal
A, b = entrada_matriz()
P, L, U = lu_com_pivotamento_parcial(A)
x = resolver_sistema_lu(P, L, U, b)

# Exibir resultados com 8 casas decimais
print("\n✅ Matriz A:")
display(sp.Matrix(A))

print("\n🔄 Matriz de Permutação P:")
display(sp.Matrix(P))

print("\n🟩 Matriz L (8 casas decimais):")
display(sp.Matrix(np.round(L, 8)))

print("\n🟥 Matriz U (8 casas decimais):")
display(sp.Matrix(np.round(U, 8)))

print("\n📌 Vetor solução x (8 casas decimais):")
for i, val in enumerate(x):
    print(f"x{i+1} = {val:.8f}")

# Verificação final
print("\n🔍 Verificação A * x:")
Ax = A @ x
display(sp.Matrix(np.round(Ax, 8)))
print("\n🔍 Vetor b original:")
display(sp.Matrix(np.round(b, 8)))


Digite o tamanho da matriz no formato 'n' para nxn: 3

📥 Preenchendo a matriz A:
  a11: 2
  a12: 1
  a13: -1
  a21: -3
  a22: 4
  a23: 2
  a31: 1
  a32: 2
  a33: 3

📥 Preenchendo o vetor b:
  b1: 120
  b2: -20
  b3: 90

✅ Matriz A:


Matrix([
[ 2.0, 1.0, -1.0],
[-3.0, 4.0,  2.0],
[ 1.0, 2.0,  3.0]])


🔄 Matriz de Permutação P:


Matrix([
[0.0, 1.0, 0.0],
[1.0, 0.0, 0.0],
[0.0, 0.0, 1.0]])


🟩 Matriz L (8 casas decimais):


...

Matrix([
[-3.0,        4.0,        2.0],
[ 0.0, 3.66666667, 0.33333333],
[ 0.0,        0.0, 3.36363636]])


📌 Vetor solução x (8 casas decimais):
x1 = 43.24324324
x2 = 29.45945946
x3 = -4.05405405

🔍 Verificação A * x:


Matrix([
[120.0],
[-20.0],
[ 90.0]])


🔍 Vetor b original:


Matrix([
[120.0],
[-20.0],
[ 90.0]])

# Questão 4 - Letra b

Resolvendo o sistema novamente pelo **método de fatoração LU**, utilizando 8 casas decimais, obtivemos os seguintes valores:

- $x = 43{,}24324324$
- $y = 29{,}45945946$
- $z = -4{,}05405405$

Comparando esses resultados com aqueles obtidos pelo **método da eliminação de Gauss**:

- $x = 43{,}24330000$
- $y = 29{,}45950000$
- $z = -4{,}05409000$

Observa-se uma pequena diferença a partir da quinta casa decimal. Essa variação é comum em métodos numéricos devido à acumulação de erros de arredondamento e à forma como cada algoritmo realiza os cálculos.

De modo geral, a fatoração LU pode apresentar maior estabilidade numérica em algumas situações, resultando em ligeiramente maior precisão.

Assim, embora ambos os métodos forneçam soluções muito próximas, o método LU demonstrou um resultado com precisão um pouco melhor neste caso.


# Questão 4 - Letra (f)

No contexto da rede de distribuição elétrica apresentada, as variáveis $x$, $y$ e $z$ representam a carga elétrica (em megawatts) associada a três subestações diferentes.

Os valores aproximados são:

- $x \approx 43,24 \text{ MW}$
- $y \approx 29,46 \text{ MW}$
- $z \approx -4,05 \text{ MW}$

Interpretando esses resultados:

- A subestação $x$ está fornecendo cerca de 43,24 MW para a rede;
- A subestação $y$ está fornecendo cerca de 29,46 MW;
- A subestação $z$ apresenta um valor negativo, indicando que ela está consumindo ou absorvendo aproximadamente 4,05 MW da rede.

Ou seja, as subestações $x$ e $y$ são fontes líquidas de energia, enquanto a subestação $z$ atua como um ponto de demanda líquida, consumindo energia para equilibrar o sistema.
